In [1]:
import random

class MinHash:
    def __init__(self, num_perm):
        self.num_perm = num_perm
        self.permutations = self._generate_permutations()
        self.signature = [float('inf')] * num_perm

    def _generate_permutations(self):
        max_hash = (1 << 32) - 1
        return [(random.randint(1, max_hash), random.randint(0, max_hash)) for _ in range(self.num_perm)]

    def update(self, item):
        item_hash = hash(item)
        for i, (a, b) in enumerate(self.permutations):
            permuted_hash = (a * item_hash + b) % (1 << 32)
            if permuted_hash < self.signature[i]:
                self.signature[i] = permuted_hash

    def get_signature(self):
        return self.signature


class MinHashLSH:
    def __init__(self, num_hashes, threshold=0.5):
        self.num_hashes = num_hashes
        self.threshold = threshold
        self.hash_tables = [dict() for _ in range(num_hashes)]

    def _get_bands(self, minhash_signature):
        bands = []
        rows_per_band = self.num_hashes // len(self.hash_tables)
        for i in range(len(self.hash_tables)):
            start = i * rows_per_band
            end = (i + 1) * rows_per_band
            band = tuple(minhash_signature[start:end])
            bands.append(band)
        return bands

    def insert(self, key, minhash_signature):
        bands = self._get_bands(minhash_signature)
        for i, band in enumerate(bands):
            if band not in self.hash_tables[i]:
                self.hash_tables[i][band] = []
            self.hash_tables[i][band].append(key)

    def query(self, minhash_signature):
        bands = self._get_bands(minhash_signature)
        candidates = set()
        for i, band in enumerate(bands):
            if band in self.hash_tables[i]:
                candidates.update(self.hash_tables[i][band])
        return list(candidates)

    def show_signatures(self, display=100):
        signatures = {}
        for i, table in enumerate(self.hash_tables):
            for band, keys in table.items():
                for key in keys:
                    if key not in signatures:
                        signatures[key] = []
                    signatures[key].append(band)
                    if len(signatures) >= display:
                        break
                if len(signatures) >= display:
                    break
            if len(signatures) >= display:
                break

        # Print only the first 10 unique keys and their bands
        for key, bands in list(signatures.items())[:10]:
            band_strings = ', '.join(str(band) for band in bands)
            print(f"Key: {key}, Bands: [{band_strings}]")

# Example usage
if __name__ == "__main__":
    # Example dataset
    movies = {
        "movie1": {"action", "adventure", "comedy"},
        "movie2": {"drama", "romance", "comedy"},
        "movie3": {"action", "thriller", "crime"},
        "movie4": {"romance", "drama"},
        "movie5": {"action", "adventure", "thriller"}
    }

    # Initialize MinHashLSH
    num_hashes = 128
    lsh = MinHashLSH(num_hashes=num_hashes)

    # Insert movies into the LSH
    for title, tags in movies.items():
        minhash = MinHash(num_perm=num_hashes)
        for tag in tags:
            minhash.update(tag)
        lsh.insert(title, minhash.get_signature())

    # Query a movie
    query_movie = {"action", "thriller", "crime"}
    minhash_query = MinHash(num_perm=num_hashes)
    for tag in query_movie:
        minhash_query.update(tag)

    similar_movies = lsh.query(minhash_query.get_signature())
    print(f"Movies similar to the query: {similar_movies}")

    # Show signatures for the first 10 items
    lsh.show_signatures()


Movies similar to the query: []
Key: movie1, Bands: [(1419423285,), (1570975987,), (289118617,), (1378591658,), (1977843934,), (289837101,), (1139614800,), (1002377611,), (202468783,), (1257156612,), (1300095073,), (1391541689,), (1305433347,), (2581390584,), (151736102,), (767202786,), (2326929810,), (9349771,), (69178213,), (77774571,), (890399364,), (3176164944,), (1386616634,), (1672327972,), (2302485203,), (1156976199,), (2710430434,), (509993037,), (809451176,), (1244640779,), (464899228,), (1447131958,), (1467821315,), (193039787,), (307211895,), (985650832,), (57636777,), (383213916,), (2126554068,), (1281232208,), (158275272,), (73874526,), (1371089515,), (1971799301,), (1142159334,), (497891320,), (1481910826,), (3021510948,), (437225180,), (1312421135,), (925173097,), (162320946,), (1476486825,), (1807287827,), (56453660,), (374417527,), (2359190966,), (684028404,), (193539344,), (2648701985,), (601192102,), (2212825169,), (2015326183,), (416529904,), (479424183,), (24904382

In [14]:
import time
import hashlib
from datasketch import MinHash as DatasketchMinHash, MinHashLSH as DatasketchMinHashLSH

# Original LSH implementation using a MinHash library
def original_lsh(movies, num_hashes=128):
    start = time.time()
    lsh = DatasketchMinHashLSH(threshold=0.5, num_perm=num_hashes)

    for title, tags in movies.items():
        m = DatasketchMinHash(num_perm=num_hashes)
        for tag in tags:
            m.update(tag.encode('utf8'))
        lsh.insert(title, m)  

    end = time.time()
    return lsh, end - start

# Custom sequential MinHashLSH implementation
class CustomMinHash:
    def __init__(self, num_perm):
        self.num_perm = num_perm
        self.signature = [float('inf')] * num_perm
    
    def update(self, tag):
        # Sử dụng SHA-1 để tạo các giá trị băm
        tag_bytes = tag.encode('utf8')
        for i in range(self.num_perm):
            hash_value = int(hashlib.sha1(tag_bytes + i.to_bytes(2, byteorder='big')).hexdigest(), 16)
            self.signature[i] = min(self.signature[i], hash_value)
    
    def get_signature(self):
        return self.signature

class CustomMinHashLSH:
    def __init__(self, num_hashes):
        self.num_hashes = num_hashes
        self.hash_tables = [{} for _ in range(num_hashes)]
    
    def _get_bands(self, minhash):
        bands = []
        rows_per_band = self.num_hashes // len(self.hash_tables)
        for i in range(len(self.hash_tables)):
            start = i * rows_per_band
            end = (i + 1) * rows_per_band
            band = tuple(minhash[start:end])
            bands.append(band)
        return bands
    
    def insert(self, key, minhash):
        bands = self._get_bands(minhash)
        for i, band in enumerate(bands):
            if band not in self.hash_tables[i]:
                self.hash_tables[i][band] = []
            self.hash_tables[i][band].append(key)
    
    def query(self, minhash):
        bands = self._get_bands(minhash)
        candidates = set()
        for i, band in enumerate(bands):
            if band in self.hash_tables[i]:
                candidates.update(self.hash_tables[i][band])
        return list(candidates)

# Custom LSH implementation
def custom_lsh(movies, num_hashes=128):
    start = time.time()
    lsh = CustomMinHashLSH(num_hashes=num_hashes)

    for title, tags in movies.items():
        minhash = CustomMinHash(num_perm=num_hashes)
        for tag in tags:
            minhash.update(tag)
        lsh.insert(title, minhash.get_signature())

    end = time.time()
    return lsh, end - start

# Function to compare the accuracy of both implementations
def compare_accuracy(original_lsh, custom_lsh, query_set, num_hashes=128):
    original_results = []
    custom_results = []
    
    for query_tags in query_set:
        # Original LSH query
        original_query_minhash = DatasketchMinHash(num_perm=num_hashes)
        for tag in query_tags:
            original_query_minhash.update(tag.encode('utf8'))
        original_result = original_lsh.query(original_query_minhash)  # Query using MinHash object
        original_results.append(set(original_result))
        
        # Custom LSH query
        custom_query_minhash = CustomMinHash(num_perm=num_hashes)
        for tag in query_tags:
            custom_query_minhash.update(tag)
        custom_result = custom_lsh.query(custom_query_minhash.get_signature())  # Query using signature list
        custom_results.append(set(custom_result))
    
    # Compare the results
    total_queries = len(query_set)
    matching_queries = sum(1 for i in range(total_queries) if original_results[i] == custom_results[i])
    accuracy = matching_queries / total_queries
    
    return accuracy

# Example usage
if __name__ == "__main__":
    # Example dataset
    movies = {
        "movie1": {"action", "adventure", "comedy"},
        "movie2": {"drama", "romance", "comedy"},
        "movie3": {"action", "thriller", "crime"},
        "movie4": {"romance", "drama"},
        "movie5": {"action", "adventure", "thriller"}
    }

    # Define a set of queries for testing accuracy
    query_set = [
        {"action", "thriller", "crime"},
        {"romance", "drama"},
        {"comedy", "adventure"},
    ]

    # Run original LSH implementation
    original_lsh_instance, original_time = original_lsh(movies, num_hashes=128)
    
    # Run custom sequential LSH implementation
    custom_lsh_instance, custom_time = custom_lsh(movies, num_hashes=128)
    
    # Compare accuracy
    accuracy = compare_accuracy(original_lsh_instance, custom_lsh_instance, query_set, num_hashes=128)

    # Output the results
    print(f"Original LSH Execution Time: {original_time:.6f} seconds")
    print(f"Custom LSH Execution Time: {custom_time:.6f} seconds")
    print(f"Accuracy between implementations: {accuracy * 100:.2f}%")


Original LSH Execution Time: 0.031327 seconds
Custom LSH Execution Time: 0.004966 seconds
Accuracy between implementations: 33.33%
